# Assignment03. Exploring Urban Mobility with Spatial Data

## Load the Packages

In [36]:
!pip install fiona
!pip install geopandas
!pip install sodapy -q
!pip install adjustText
!pip install plotly --upgrade
!pip install mapclassify
!pip install chart-studio

In [37]:
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# Load a required package 
# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import chart_studio.plotly as py
import chart_studio.tools as tls
import os
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape

# A magic functin that renders the figure in a notebook 
%matplotlib inline 

from matplotlib import pyplot as plt

#Chart Studio API
import chart_studio
username='bkirthi'
api_key='PHHYaOZKZDA7BF3nUUtO'
chart_studio.tools.set_credentials_file(username=username,
                                        api_key=api_key)



## Get Community Area Boundaries

In [38]:
!git clone https://github.com/kirthi-b/Chicago-Rideshare

fatal: destination path 'Chicago-Rideshare' already exists and is not an empty directory.


In [39]:
gdf = gpd.read_file("/content/Chicago-Rideshare/Data/CA_Boundaries/", driver = "shapefile")

gdf.head()

,area,area_num_1,area_numbe,comarea,comarea_id,community,perimeter,shape_area,shape_len,geometry
0,0.0,35,35,0.0,0.0,DOUGLAS,0.0,4.600462e+07,31027.054510,"POLYGON ((-87.60914 41.84469, -87.60915 41.844..."
1,0.0,36,36,0.0,0.0,OAKLAND,0.0,1.691396e+07,19565.506153,"POLYGON ((-87.59215 41.81693, -87.59231 41.816..."
2,0.0,37,37,0.0,0.0,FULLER PARK,0.0,1.991670e+07,25339.089750,"POLYGON ((-87.62880 41.80189, -87.62879 41.801..."
3,0.0,38,38,0.0,0.0,GRAND BOULEVARD,0.0,4.849250e+07,28196.837157,"POLYGON ((-87.60671 41.81681, -87.60670 41.816..."
4,0.0,39,39,0.0,0.0,KENWOOD,0.0,2.907174e+07,23325.167906,"POLYGON ((-87.59215 41.81693, -87.59215 41.816..."


## Get Rideshare Data

In [40]:
API = 'https://data.cityofchicago.org/resource/m6dm-c72p.csv?'

In [54]:
query = {'$select': 'trip_start_timestamp, trip_seconds, trip_miles, pickup_community_area, pickup_centroid_latitude, pickup_centroid_longitude',
         '$where': 'trip_start_timestamp >= "2020-10-01" and  trip_start_timestamp < "2021-11-01" and pickup_community_area IS NOT NULL ',
         '$order': 'trip_start_timestamp',
         '$limit': 500} #change to 42000000 later

rs_df = pd.read_csv(API + urlencode(query))
rs_df.head()

,trip_start_timestamp,trip_seconds,trip_miles,pickup_community_area,pickup_centroid_latitude,pickup_centroid_longitude
0,2020-10-01T00:00:00.000,798,3.6284,38,41.812949,-87.617860
1,2020-10-01T00:00:00.000,1895,7.1873,7,41.922686,-87.649489
2,2020-10-01T00:00:00.000,247,1.0866,6,41.944227,-87.655998
3,2020-10-01T00:00:00.000,787,3.6098,43,41.761578,-87.572782
4,2020-10-01T00:00:00.000,768,4.9600,5,41.947792,-87.683835


## Get Median Income Data

In [5]:
client = Socrata("data.cityofchicago.org", None)
results = client.get("igwz-8jzy", limit=2000)
ca_df = pd.DataFrame.from_records(results)